# Integration

Programmatically

**Input:** Text and PDF

**Intermediate:**

- Scene generation (Text)
- Segmentation of the scene
- Separate scene into multiple images
- Audio
- Music

**Output:** Video (images) with audio


## Setup


In [ ]:
# First-time setup:
# https://cloud.google.com/text-to-speech/docs/create-audio-text-client-libraries#client-libraries-install-python
# gcloud init
# poetry init
# poetry add google-cloud-texttospeech

## Constants


In [34]:
GCP_PROJECT_ID = "bliss-hack25fra-9587"
BUCKET_NAME = "bliss-hack25fra-9587-eu-notebooks"

In [28]:
PDF_FILE_NAME = "BDA_PowerLine_WebCast3.pdf"
TEXT_AND_PDF_TO_SCENES_PROMPT = """Erstelle ein Skript für PowerLine webcast3 ein absurd-lustiges TikTok-Video basierend auf der Bedienungsanleitung. Der Sprecher spricht in einer neutralen, männlichen Stimme. Es soll eine kurze unterhaltsame Geschichte sein. Der Humor in dem Video besteht darin, die spezifischen Worte aus der Bedienungsanleitung in absurden Situationen wieder zu finden. Die Geschichte soll interessant, unterhaltsam und random sein. Alles ist sehr satirisch aufgeladen. Das Video ist insgesamt sehr kurz, so 10-20s. Der Text ist wichtiger als die visuellen Elemente. Der Sprecher kann seine Stimme nicht verstellen. Der erste Satz ist der wichtigste. Er soll catchy sein und direkt in die situation einsteigen, keine einleitung und kein Schluss. Die Geschichte geht schnell vorran, wenige Floskeln"""

## Retrieve PDF from Google Cloud Storage

Returns 64-bit encoded PDF file from Google Cloud Storage.


In [ ]:
import base64
from google.cloud import storage

# Instantiates a client
storage_client = storage.Client()

# The name for the bucket
bucket_name = BUCKET_NAME

# Get bucket
bucket = storage_client.bucket(bucket_name)

# Specify the blob name
blob_name = PDF_FILE_NAME

# Get blob
blob = bucket.blob(blob_name)

# Download as bytes and encode to base64
content = blob.download_as_bytes()
base64_content = base64.b64encode(content).decode("utf-8")

print(f"Downloaded and encoded {blob_name}")

Downloaded and encoded BDA_PowerLine_WebCast3.pdf


## Text-and-PDF-to-Text


In [ ]:
# Setup
# pip install --upgrade google-genai
# gcloud auth application-default login

from google import genai
from google.genai import types
import base64


def generate():
    client = genai.Client(
        vertexai=True,
        project=GCP_PROJECT_ID,
        location="global",
    )

    document1 = types.Part.from_bytes(
        data=base64.b64decode(f"""{base64_content}"""),
        mime_type="application/pdf",
    )
    text1 = types.Part.from_text(text=TEXT_AND_PDF_TO_SCENES_PROMPT)

    model = "gemini-2.0-flash-001"
    contents = [types.Content(role="user", parts=[document1, text1])]
    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        max_output_tokens=8192,
        response_modalities=["TEXT"],
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(
                category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"
            ),
            types.SafetySetting(
                category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"
            ),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        ],
    )

    text_output = ""
    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        text_output += chunk.text
        print(chunk.text, end="")
    return text_output


script_text = generate()

Absolut! Hier ist ein Skript für ein satirisches TikTok-Video, das auf der Bedienungsanleitung des PowerLine Webcast 3 basiert:

**Titel:** Wenn deine Bedienungsanleitung ein Drehbuchautor wäre...

**Visuelles:**

*   **Szene 1:** (0-2 Sek) Ein Mann, perfekt angezogen, steht inmitten eines völlig chaotischen Raumes. Überall Kabel, Geräte, Pizzakartons. Er hält den PowerLine Webcast 3 Adapter.
*   **Szene 2:** (3-4 Sek) Nahaufnahme des Adapters. Text-Overlay: "Stecken Sie den Adapter in die Steckdose".
*   **Szene 3:** (5-7 Sek) Der Mann steckt den Adapter ein. Plötzlich beginnt das Chaos um ihn herum zu tanzen. Text-Overlay: "Moduliert das Signal".
*   **Szene 4:** (8-10 Sek) Schnitt zu einem anderen Raum, wo eine Katze auf einem Laptop sitzt und tippt. Text-Overlay: "Datenübertragung".
*   **Szene 5:** (11-13 Sek) Zurück zum Mann. Er blickt direkt in die Kamera, mit einem Gesichtsausdruck zwischen Verwirrung und Akzeptanz. Text-Overlay: "Funktionsweise".
*   **Szene 6:** (14-16 Sek) S

## Text-to-Speech


In [32]:
"""Synthesizes speech from the input string of text or ssml.
Make sure to be working in a virtual environment.

Note: ssml must be well-formed according to:
    https://www.w3.org/TR/speech-synthesis/
"""
from google.cloud import texttospeech

# Instantiates a client
client = texttospeech.TextToSpeechClient()

# Set the text input to be synthesized
synthesis_input = texttospeech.SynthesisInput(
    text="Hello! Today I would like to talk to you about TechniSat"
)

# Build the voice request, select the language code ("en-US") and the ssml
# voice gender ("neutral")
voice = texttospeech.VoiceSelectionParams(
    language_code="en-US", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
)

# Select the type of audio file you want returned
audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)

# Perform the text-to-speech request on the text input with the selected
# voice parameters and audio file type
response = client.synthesize_speech(
    input=synthesis_input, voice=voice, audio_config=audio_config
)

# The response's audio_content is binary.
with open("output.mp3", "wb") as out:
    # Write the response to the output file.
    out.write(response.audio_content)
    print('Audio content written to file "output.mp3"')

Audio content written to file "output.mp3"


In [2]:
import cv2
import os
import subprocess
import imageio_ffmpeg
import ffmpeg

def generate_video(image_folder = 'data/images', audio_file = './data/synthesis.wav'):
    os.remove('./finished_video.mp4')
    os.remove('./video.avi')
    os.remove('./video.mp4')

    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter('./video.avi', 0, 1, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    #cv2.destroyAllWindows()
    video.release()



    ffmpeg_exe = imageio_ffmpeg.get_ffmpeg_exe()
    subprocess.run([
        ffmpeg_exe,
        '-y',               # overwrite output if exists
        '-i', 'video.avi',  # input video
        '-c:v', 'libx264',  # video codec
        '-c:a', 'aac',      # audio codec
        '-shortest',        # finish when the shorter stream ends
        'video.mp4'         # output file
    ])

    input_video = ffmpeg.input('./video.mp4')

    input_audio = ffmpeg.input(audio_file)

    ffmpeg.concat(input_video, input_audio, v=1, a=1).output('finished_video.mp4').run(cmd=ffmpeg_exe)
    os.remove('./video.avi')
    os.remove('./video.mp4')